In [281]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler

# Feature Engineering

First, we will load the data into our DataFrame. Sort by date and denote which Season each match corresponds to. 

In [282]:
data_dir = "finalData.csv"
df = pd.read_csv(data_dir)

# renaming the columns
df.columns = ['Date', 'Home Team', 'Away Team', 'Full Time Home Goals', 'Full Time Away Goals', 'Full Time Result',       
'Half Time Home Goals', 'Half Time Away Goals', 'Half Time Result', 'Referee', 'Home Shots', 'Away Shots', 'Home Shots on Target',   
'Away Shots on Target', 'Home Corners', 'Away Corners', 'Home Fouls', 'Away Fouls', 'Home Yellow Cards', 'Away Yellow Cards',     
'Home Red Cards', 'Away Red Cards', 'Home Possession', 'Away Possession', 'Home Passes Completed', 'Home Passes PCT',
'Home Progressive Passes', 'Home Progressive Passing Distance', 'Home xG', 'Home Take Ons Won', 'Home Take Ons', 
'Home Interceptions', 'Home Blocks', 'Home Touches', 'Home Touches Def 3rd', 'Home Touches Mid 3rd', 'Home Touches Att 3rd',
'Home Carries', 'Home Carries Progressive Distance', 'Home Tackles', 'Home Tackles Won', 'Away Passes Completed',
'Away Passes PCT', 'Away Progressive Passes', 'Away Progressive Passing Distance', 'Away xG', 'Away Shots REDUNDANT', 'Away Shots on Target REDUNDANT', 
'Away Take Ons Won', 'Away Take Ons', 'Away Interceptions', 'Away Blocks', 'Away Touches', 'Away Touches Def 3rd',
'Away Touches Mid 3rd', 'Away Touches Att 3rd', 'Away Carries', 'Away Carries Progressive Distance', 'Away Tackles',
'Away Tackles Won']

df.drop(columns=['Away Shots on Target REDUNDANT', 'Away Shots REDUNDANT'], inplace=True)

In [283]:
# ensure sorted by date
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df = df.sort_values(by='Date')

# each season starts at 08 and ends at 05 of next year - 2000-2001 season will be the 2000 season
def get_season(date):
    if date.month >= 8:  
        return (date.year)
    else:  
        return (date.year - 1)

df['Season'] = df['Date'].apply(get_season)

Here, we begin engineering new features. The following function enables us to find the previous matches a team played (within this or the previous season). Using this key function, we created several features:

Average Goal Conversion Rate Difference, Average Attacking Intensity Difference, Average Disciplinary Pressure Difference, Recent Performances, Average Goals Scored Difference, Average Goals Conceded Difference, ...

Note: All averages use the previous RECENCY_NUM matches. If the number of previous matches in this season is too few, it will draw from the previous season as well. 

Note 2: After feature engineering, we will have to remove 101 matches that do not have RECENCY_NUM=5 matches prior. Unfortunately, this will reduce our dataset by 3.66%.

In [284]:
RECENCY_NUM = 5

# returns -1 if number of previous matches < RECENCY_NUM
# returns dataframe consisting of the previous 
def get_recent_matches(team, current_season, current_date, last_x_games=RECENCY_NUM):
    # filter match for specific team + current and previous season
    team_matches = df[((df['Home Team'] == team) | (df['Away Team'] == team)) & ((df['Season'] == current_season) | (df['Season'] == current_season - 1))]
    
    # remove dates following the match
    team_matches = team_matches[((team_matches['Date'] < current_date) & (team_matches['Season'] == current_season)) | (team_matches['Season'] == current_season - 1)]
    
    if len(team_matches) < last_x_games:
        return team_matches.head(0)  # returns an empty dataframe
    
    team_matches.sort_values(by='Date')
    return team_matches.tail(last_x_games)

In [285]:
# create GOAL CONVERSION RATE, ATTACKING INTENSITY, DISCIPLINARY PRESSURE per individual match
def original_gcr_ai_dp(df):
    df['Home Goal Conversion Rate'] = df['Full Time Home Goals'] / df['Home Shots on Target'].replace(0, 1)
    df['Away Goal Conversion Rate'] = df['Full Time Away Goals'] / df['Away Shots on Target'].replace(0, 1)
    
    df['Home Attacking Intensity'] = df['Home Shots'] + df['Home Corners']
    df['Away Attacking Intensity'] = df['Away Shots'] + df['Away Corners']
    
    df['Home Disciplinary Pressure'] = df['Home Fouls'] + df['Home Yellow Cards'] + df['Home Red Cards']
    df['Away Disciplinary Pressure'] = df['Away Fouls'] + df['Away Yellow Cards'] + df['Away Red Cards']
    
    return df
    
# sets up the recency average columns
def gcr_ai_dp_recency_setup(df):
    df['Avg Home Goal Conversion Rate (Recency)'] = 0.0
    df['Avg Away Goal Conversion Rate (Recency)'] = 0.0
    
    df['Avg Home Attacking Intensity (Recency)'] = 0.0
    df['Avg Away Attacking Intensity (Recency)'] = 0.0
    
    df['Avg Home Disciplinary Pressure (Recency)'] = 0.0
    df['Avg Away Disciplinary Pressure (Recency)'] = 0.0
    
    return df

def compute_differences_gcr_ai_dp(df):
    df['Avg Goal Conversion Rate Difference (Recency)'] = 0.0
    df['Avg Attacking Intensity Difference (Recency)'] = 0.0
    df['Avg Disciplinary Pressure Difference (Recency)'] = 0.0
    
    for idx, row in df.iterrows():
        # Goal Conversion Rate
        if row['Avg Home Goal Conversion Rate (Recency)'] == -1 or row['Avg Away Goal Conversion Rate (Recency)'] == -1:
            df.at[idx, 'Avg Goal Conversion Rate Difference (Recency)'] = -1
        else:
            df.at[idx, 'Avg Goal Conversion Rate Difference (Recency)'] = row['Avg Home Goal Conversion Rate (Recency)'] - row['Avg Away Goal Conversion Rate (Recency)']
            
        # Attacking Intensity
        if row['Avg Home Attacking Intensity (Recency)'] == -1 or row['Avg Away Attacking Intensity (Recency)'] == -1:
            df.at[idx, 'Avg Attacking Intensity Difference (Recency)'] = -1
        else:
            df.at[idx, 'Avg Attacking Intensity Difference (Recency)'] = row['Avg Home Attacking Intensity (Recency)'] - row['Avg Away Attacking Intensity (Recency)']
        
        # Disciplinary Pressure
        if row['Avg Home Disciplinary Pressure (Recency)'] == -1 or row['Avg Away Disciplinary Pressure (Recency)'] == -1:
            df.at[idx, 'Avg Disciplinary Pressure Difference (Recency)'] = -1
        else:
            df.at[idx, 'Avg Disciplinary Pressure Difference (Recency)'] = row['Avg Home Disciplinary Pressure (Recency)'] - row['Avg Away Disciplinary Pressure (Recency)']
    
    return df
    
    
    
# create an avg GOAL CONVERSION RATE, an avg ATTACKING INTENSITY, an avg DISCIPLINARY PRESSURE
def gcr_ai_dp_recency(df, last_x_games=RECENCY_NUM):
    df = original_gcr_ai_dp(df)
    df = gcr_ai_dp_recency_setup(df)
    
    for idx, row in df.iterrows():
        current_season = row['Season']
        current_date = row['Date']
        home_team = row['Home Team']
        away_team = row['Away Team']
        
        home_team_matches = get_recent_matches(home_team, current_season, current_date, last_x_games)
        away_team_matches = get_recent_matches(away_team, current_season, current_date, last_x_games)
        
        # home team matches
        if home_team_matches.empty:
            df.at[idx, 'Avg Home Goal Conversion Rate (Recency)'] = -1
            df.at[idx, 'Avg Home Attacking Intensity (Recency)'] = -1
            df.at[idx, 'Avg Home Disciplinary Pressure (Recency)'] = -1
        else:
            avgGC, avgAI, avgDP = 0, 0, 0
            for _, row2 in home_team_matches.iterrows():
                if row2['Home Team'] == home_team:
                    avgGC += row2['Home Goal Conversion Rate']
                    avgAI += row2['Home Attacking Intensity']
                    avgDP += row2['Home Disciplinary Pressure']
                else:
                    avgGC += row2['Away Goal Conversion Rate']
                    avgAI += row2['Away Attacking Intensity']
                    avgDP += row2['Away Disciplinary Pressure']
            df.at[idx, 'Avg Home Goal Conversion Rate (Recency)'] = avgGC / len(home_team_matches)
            df.at[idx, 'Avg Home Attacking Intensity (Recency)'] = avgAI / len(home_team_matches)
            df.at[idx, 'Avg Home Disciplinary Pressure (Recency)'] = avgDP / len(home_team_matches)
            
        # away team matches
        if away_team_matches.empty:
            df.at[idx, 'Avg Away Goal Conversion Rate (Recency)'] = -1
            df.at[idx, 'Avg Away Attacking Intensity (Recency)'] = -1
            df.at[idx, 'Avg Away Disciplinary Pressure (Recency)'] = -1
        else:
            avgGC, avgAI, avgDP = 0, 0, 0
            for _, row2 in away_team_matches.iterrows():
                if row2['Home Team'] == away_team:
                    avgGC += row2['Home Goal Conversion Rate']
                    avgAI += row2['Home Attacking Intensity']
                    avgDP += row2['Home Disciplinary Pressure']
                else:
                    avgGC += row2['Away Goal Conversion Rate']
                    avgAI += row2['Away Attacking Intensity']
                    avgDP += row2['Away Disciplinary Pressure']
            df.at[idx, 'Avg Away Goal Conversion Rate (Recency)'] = avgGC / len(away_team_matches)
            df.at[idx, 'Avg Away Attacking Intensity (Recency)'] = avgAI / len(away_team_matches)
            df.at[idx, 'Avg Away Disciplinary Pressure (Recency)'] = avgDP / len(away_team_matches)
            
    df = compute_differences_gcr_ai_dp(df)        
    
    return df

df = gcr_ai_dp_recency(df)
# df[df['Avg Home Goal Conversion Rate (Recency)'] == -1].shape[0]
# returns the number of -1s: 101 for Home, 92 for Away, same for all 3 versions of paired columns.

In [286]:
# creates a column of recent winning streaks for each home and away team
def win_streak(df, last_x_games=RECENCY_NUM):
    df['Match Outcome'] = df['Full Time Result'].map({'H': 1, 'D': 0, 'A': -1})
    
    # finds the recent wins
    def get_recent_performance(team, current_season, current_date, last_x_games):
        recent_matches = get_recent_matches(team, current_season, current_date, last_x_games)
        
        # if number of previous games < last_x_games
        if recent_matches.empty:
            return -1
        
        # after filtering, counts the number of recent wins in a row
        wins = 0
        for _, row in recent_matches.iterrows():
            if row['Home Team'] == team and row['Match Outcome'] == 1:
                wins += 1
            elif row['Home Team'] == team and (row['Match Outcome'] == 0 or row['Match Outcome'] == -1):
                wins = 0
            elif row['Away Team'] == team and row['Match Outcome'] == -1:
                wins += 1
            elif row['Away Team'] == team and (row['Match Outcome'] == 0 or row['Match Outcome'] == 1):
                wins = 0

        return wins

    for idx, row in df.iterrows():
        current_season = row['Season']
        current_date = row['Date']
        home_team = row['Home Team']
        away_team = row['Away Team']
        
        home_performance = get_recent_performance(home_team, current_season, current_date, last_x_games)
        away_performance = get_recent_performance(away_team, current_season, current_date, last_x_games)

        df.loc[idx, 'Recent Performance Home Team'] = home_performance
        df.loc[idx, 'Recent Performance Away Team'] = away_performance
    return df

df = win_streak(df)
# df[df['Recent Performance Home Team'] == -1].shape[0]     returns number of rows w/ -1, currently 101 for Home, 92 for Away
# df = df[df['Recent Performance Home Team'] != -1]         drops the rows w/ -1

In [287]:
# creates new columns for avg scores/conceeds for home/away teams, per match
def avg_goals_scored_conceded_last_x_games(df, x=RECENCY_NUM):
    # recency average scores + conceeds
    def calculate_avgs(team, current_season, match_date, x):
        recent_matches = get_recent_matches(team, current_season, match_date, x)
        
        if recent_matches.empty:
            return -1, -1
        
        goals_scored, goals_conceded = 0, 0

        for _, row in recent_matches.iterrows():
            if row['Home Team'] == team:
                goals_scored += row['Full Time Home Goals']
                goals_conceded += row['Full Time Away Goals']
            elif row['Away Team'] == team:
                goals_scored += row['Full Time Away Goals']
                goals_conceded += row['Full Time Home Goals']

        num_matches = len(recent_matches)
        avg_scored = goals_scored / num_matches
        avg_conceded = goals_conceded / num_matches

        return avg_scored, avg_conceded

    df['Avg Goals Scored Home (Recency)'] = 0.0
    df['Avg Goals Conceded Home (Recency)'] = 0.0
    df['Avg Goals Scored Away (Recency)'] = 0.0
    df['Avg Goals Conceded Away (Recency)'] = 0.0

    running_stats = {}

    for idx, row in df.iterrows():
        for team_type in ['Home Team', 'Away Team']:
            team = row[team_type]
            current_season = row['Season']
            match_date = row['Date']

            if current_season not in running_stats:
                running_stats[current_season] = {}

            if team not in running_stats[current_season]:
                running_stats[current_season][team] = {
                    'scored': [],
                    'conceded': []
                }

            team_stats = running_stats[current_season][team]
            avg_scored, avg_conceded = calculate_avgs(team, current_season, match_date, x)

            if team_type == 'Home Team':
                df.at[idx, 'Avg Goals Scored Home (Recency)'] = avg_scored
                df.at[idx, 'Avg Goals Conceded Home (Recency)'] = avg_conceded
            else:
                df.at[idx, 'Avg Goals Scored Away (Recency)'] = avg_scored
                df.at[idx, 'Avg Goals Conceded Away (Recency)'] = avg_conceded

    return df

def avg_goals_difference(df):
    df['Avg Goals Scored Difference (Recency)'] = 0.0
    df['Avg Goals Conceded Difference (Recency)'] = 0.0
    
    for idx, row in df.iterrows():
        # goals scored
        if row['Avg Goals Scored Home (Recency)'] == -1 or row['Avg Goals Scored Away (Recency)'] == -1:
            df.at[idx, 'Avg Goals Scored Difference (Recency)'] = -1
        else:
            df.at[idx, 'Avg Goals Scored Difference (Recency)'] = row['Avg Goals Scored Home (Recency)'] - row['Avg Goals Scored Away (Recency)']
            
        # goals conceded:
        if row['Avg Goals Conceded Home (Recency)'] == -1 or row['Avg Goals Conceded Away (Recency)'] == -1:
            df.at[idx, 'Avg Goals Conceded Difference (Recency)'] = -1
        else:
            df.at[idx, 'Avg Goals Conceded Difference (Recency)'] = row['Avg Goals Conceded Home (Recency)'] - row['Avg Goals Conceded Away (Recency)']
    
    return df
            

df = avg_goals_scored_conceded_last_x_games(df)
df = avg_goals_difference(df)
# df[df['Avg Goals Scored Home (Recency)'] == -1].shape[0]     returns the number of -1s, 101 for Home, 92 for Away
# df[df['Avg Goals Scored Difference (Recency)'] == -1].shape[0] IS A BAD IDEA, DIFFERENCES CAN ACTUALLY EQUAL -1

In [288]:
# Team Points = total points in the game. win:3, draw:1, loss:0... DROPPED AFTERWARDS
# Total Points = cumulative sum of the team points, per team, per season
# Point Difference = difference between total points of the teams, per match
def calculate_season_points(df):
    df['Home Team Points'] = 0
    df['Away Team Points'] = 0

    for idx, row in df.iterrows():
        if row['Match Outcome'] == 1:  
            df.at[idx, 'Home Team Points'] = 3
            df.at[idx, 'Away Team Points'] = 0
        elif row['Match Outcome'] == 0: 
            df.at[idx, 'Home Team Points'] = 1
            df.at[idx, 'Away Team Points'] = 1
        elif row['Match Outcome'] == -1:  
            df.at[idx, 'Home Team Points'] = 0
            df.at[idx, 'Away Team Points'] = 3

    df['Home Total Seasonal Points'] = (df.groupby(['Home Team', 'Season'])['Home Team Points'].cumsum())
    df['Away Total Seasonal Points'] = (df.groupby(['Away Team', 'Season'])['Away Team Points'].cumsum())
    df['Seasonal Point Difference'] = df['Home Total Seasonal Points'] - df['Away Total Seasonal Points']
    df.drop(columns=['Home Team Points', 'Away Team Points'], inplace=True)

    return df
df = calculate_season_points(df)

In [295]:
with pd.option_context('display.max_rows', 10, 'display.max_columns', None): 
    display(df)

,Date,Home Team,Away Team,Full Time Home Goals,Full Time Away Goals,Full Time Result,Half Time Home Goals,Half Time Away Goals,Half Time Result,Referee,Home Shots,Away Shots,Home Shots on Target,Away Shots on Target,Home Corners,Away Corners,Home Fouls,Away Fouls,Home Yellow Cards,Away Yellow Cards,Home Red Cards,Away Red Cards,Home Possession,Away Possession,Home Passes Completed,Home Passes PCT,Home Progressive Passes,Home Progressive Passing Distance,Home xG,Home Take Ons Won,Home Take Ons,Home Interceptions,Home Blocks,Home Touches,Home Touches Def 3rd,Home Touches Mid 3rd,Home Touches Att 3rd,Home Carries,Home Carries Progressive Distance,Home Tackles,Home Tackles Won,Away Passes Completed,Away Passes PCT,Away Progressive Passes,Away Progressive Passing Distance,Away xG,Away Take Ons Won,Away Take Ons,Away Interceptions,Away Blocks,Away Touches,Away Touches Def 3rd,Away Touches Mid 3rd,Away Touches Att 3rd,Away Carries,Away Carries Progressive Distance,Away Tackles,Away Tackles Won,Season,Home Goal Conversion Rate,Away Goal Conversion Rate,Home Attacking Intensity,Away Attacking Intensity,Home Disciplinary Pressure,Away Disciplinary Pressure,Avg Home Goal Conversion Rate (Recency),Avg Away Goal Conversion Rate (Recency),Avg Home Attacking Intensity (Recency),Avg Away Attacking Intensity (Recency),Avg Home Disciplinary Pressure (Recency),Avg Away Disciplinary Pressure (Recency),Avg Goal Conversion Rate Difference (Recency),Avg Attacking Intensity Difference (Recency),Avg Disciplinary Pressure Difference (Recency),Match Outcome,Recent Performance Home Team,Recent Performance Away Team,Avg Goals Scored Home (Recency),Avg Goals Conceded Home (Recency),Avg Goals Scored Away (Recency),Avg Goals Conceded Away (Recency),Avg Goals Scored Difference (Recency),Avg Goals Conceded Difference (Recency),Home Total Seasonal Points,Away Total Seasonal Points,Seasonal Point Difference
0,2017-08-11,Arsenal,Leicester City,4.0,3.0,H,2.0,2.0,D,M Dean,27.0,6.0,10.0,3.0,9.0,4.0,9.0,12.0,0.0,1.0,0.0,0.0,68,32,565,83.0,62,3216,2.6,16,19,13,6,801,203,355,252,519,1686,23,17,192,61.0,16,2006,1.5,5,16,11,18,427,169,142,120,150,422,17,8,2017,0.400000,1.000000,36.0,10.0,9.0,13.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3,0,3
1,2017-08-12,Brighton & Hove Albion,Manchester City,0.0,2.0,A,0.0,0.0,D,M Oliver,6.0,14.0,2.0,4.0,3.0,10.0,6.0,9.0,0.0,2.0,0.0,0.0,23,77,146,58.9,5,1685,0.3,7,9,12,10,343,177,119,51,126,366,10,8,718,87.6,87,3285,1.9,6,12,9,9,897,130,474,298,566,1779,10,6,2017,0.000000,0.500000,9.0,24.0,6.0,11.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,3,-3
2,2017-08-12,Chelsea,Burnley,2.0,3.0,A,0.0,3.0,A,C Pawson,19.0,10.0,6.0,5.0,8.0,5.0,16.0,11.0,3.0,3.0,2.0,0.0,62,38,468,82.4,48,2448,1.5,5,8,9,6,667,201,268,203,394,1440,10,8,257,73.9,26,2521,0.6,2,3,13,13,453,168,212,74,190,460,8,4,2017,0.333333,0.600000,27.0,15.0,21.0,14.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,3,-3
3,2017-08-12,Crystal Palace,Huddersfield Town,0.0,3.0,A,0.0,2.0,A,J Moss,14.0,8.0,4.0,6.0,12.0,9.0,7.0,19.0,1.0,3.0,0.0,0.0,56,45,333,75.0,33,2847,1.1,18,23,19,9,561,180,274,113,238,751,24,19,230,64.6,18,2165,1.5,5,11,9,12,460,173,166,127,167,452,28,19,2017,0.000000,0.500000,26.0,17.0,8.0,22.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,3,-3
4,2017-08-12,Everton,Stoke City,1.0,0.0,H,1.0,0.0,H,N Swarbrick,9.0,9.0,4.0,1.0,6.0,7.0,13.0,10.0,1.0,1.0,0.0,0.0,60,40,425,78.0,32,2670,0.6,1,7,19,6,670,227,308,142,313,880,17,12,233,65.4,34,2068,0.4,10,13,24,10,473,154,192,135,190,729,18,12,2017,0.250000,0.000000,15.0,16.0,14.0,11.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..